- median filter pupil
- pupil: blink_threshold, interpolation, median filter, low pass filter (maybe in that order)
- irgendetwas ließ sich auch nicht plotten für trials über 1?! -> add_blink_threshold might be the problem


In [5]:
# TODO: trial 28 is invalid, half of pupil missing -> to be removed FIXME TODO
# -> maybe trial 28 can be saved when we only only use the right eye,
# maybe this can be implemented in the final mean calculation

In [6]:
%%capture
from pathlib import Path

if Path.cwd().stem == "features":
    %cd ../..
    %load_ext autoreload
    %autoreload 2

In [8]:
import logging
from pathlib import Path

import holoviews as hv
import hvplot.polars  # noqa
import matplotlib.pyplot as plt
import numpy as np
import polars as pl
import scipy.signal as signal
from icecream import ic
from polars import col

from src.data.database_manager import DatabaseManager
from src.data.quality_checks import check_sample_rate
from src.features.pupil import (
    _get_blink_segments,
    add_blink_threshold,
    average_pupils,
    extend_periods_around_blinks,
    low_pass_filter_pupil_tonic,
    median_filter_pupil,
)
from src.features.resampling import interpolate_and_fill_nulls
from src.features.scaling import scale_min_max
from src.features.transforming import map_trials, merge_dfs
from src.features.utils import to_describe
from src.log_config import configure_logging
from src.plots.confidence_intervals import plot_confidence_intervals
from src.plots.utils import prepare_multiline_hvplot

logger = logging.getLogger(__name__.rsplit(".", maxsplit=1)[-1])
configure_logging(
    stream_level=logging.DEBUG,
    ignore_libs=["matplotlib", "Comm", "bokeh", "tornado", "param", "numba"],
)

pl.Config.set_tbl_rows(12)  # for the 12 trials
hv.output(widget_location="bottom", size=130)

In [9]:
db = DatabaseManager()

In [10]:
exclude_trials_with_measurement_problems = True
with db:
    pupil = db.get_table("Feature_Pupil", exclude_trials_with_measurement_problems)
    stimulus = db.get_table(
        "Feature_Stimulus", exclude_trials_with_measurement_problems
    )
    trials = db.get_table("Trials", exclude_trials_with_measurement_problems)


df = merge_dfs([pupil, stimulus])
df = merge_dfs(
    dfs=[df, trials],
    on=["trial_id", "participant_id", "trial_number"],
)
df = interpolate_and_fill_nulls(df)

In [11]:
pupil

trial_id,trial_number,participant_id,rownumber,timestamp,pupil_l_raw,pupil_r_raw,pupil_r,pupil_l,pupil_mean,pupil_mean_tonic
u16,u8,u8,u32,f64,f64,f64,f64,f64,f64,f64
23,1,3,11073,184810.9923,4.021351,4.492471,4.491222,4.022056,4.256639,4.309533
23,1,3,11079,184911.0835,4.042243,4.502328,4.50289,4.041127,4.272009,4.31615
23,1,3,11085,185011.2371,4.060301,4.527627,4.527639,4.061712,4.294675,4.32272
23,1,3,11091,185111.1811,4.087508,4.564853,4.5643,4.086058,4.325179,4.328876
23,1,3,11097,185211.3139,4.128739,4.582082,4.582937,4.12982,4.356379,4.334678
23,1,3,11103,185311.4403,4.187119,4.589672,4.589475,4.187547,4.388511,4.339869
…,…,…,…,…,…,…,…,…,…,…
513,12,50,161500,2.6956e6,5.227889,4.806865,4.809087,5.231606,5.020346,5.182727
513,12,50,161506,2.6957e6,5.078326,4.70844,4.709043,5.074315,4.891679,5.176408


In [12]:
df.hvplot(
    x="timestamp", y=["pupil_r", "pupil_mean", "pupil_mean_tonic"], groupby="trial_id"
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'aa4312ca-f947-41eb-9f1e-9547a1ea89a3': {'version…

In [48]:
@map_trials
def low_pass_filter_pupil(
    df: pl.DataFrame,
    sample_rate: float = SAMPLE_RATE,
    lowcut: float | None = None,
    highcut: float | None = None,
    order: int = 2,
    pupil_columns: list[str] = ["pupil_r", "pupil_l"],
) -> pl.DataFrame:
    return df.with_columns(
        pl.col(
            pupil_columns
        ).map_batches(  # map_batches to apply the filter to each column
            lambda x: butterworth_filter(
                x,
                SAMPLE_RATE,
                lowcut=lowcut,
                highcut=highcut,
                order=order,
            )
        )
    )


df = low_pass_filter_pupil(df, sample_rate=10, highcut=0.2)

In [7]:
df.select("pupil_mean").describe()

statistic,pupil_mean
str,f64
"""count""",1.681041e6
"""null_count""",0.0
"""mean""",3.740733
"""std""",0.586672
"""min""",1.898888
"""25%""",3.328454
"""50%""",3.69181
"""75%""",4.088993
"""max""",6.116632


In [8]:
with db:
    pre_pupil = db.get_table(
        "Preprocess_Pupil", exclude_trials_with_measurement_problems
    )
    pupil = db.get_table("Feature_Pupil", exclude_trials_with_measurement_problems)

In [9]:
pre_pupil

trial_id,trial_number,participant_id,rownumber,timestamp,pupil_l_raw,pupil_r_raw,pupil_r,pupil_l
u16,u8,u8,u32,f64,f64,f64,f64,f64
23,1,3,11073,184810.9923,4.068285,4.544422,4.544422,4.068285
23,1,3,11074,184827.5539,4.071086,4.546643,4.546643,4.071086
23,1,3,11075,184844.3091,4.07493,4.545161,4.545161,4.07493
23,1,3,11076,184860.9299,4.081421,4.543365,4.543365,4.081421
23,1,3,11077,184877.6867,4.080965,4.552524,4.552524,4.080965
23,1,3,11078,184894.3395,4.086535,4.555312,4.555312,4.086535
…,…,…,…,…,…,…,…,…
513,12,50,161529,2.6960e6,4.940831,4.702713,4.702713,4.940831
513,12,50,161530,2.6961e6,4.942234,4.707606,4.707606,4.942234


In [10]:
median_pupil = median_filter_pupil(pre_pupil, size_in_seconds=1)

In [33]:
median_pupil.with_columns(pupil_tonic=col("pupil_r"))

trial_id,trial_number,participant_id,rownumber,timestamp,pupil_l_raw,pupil_r_raw,pupil_r,pupil_l,pupil_tonic
u16,u8,u8,u32,f64,f64,f64,f64,f64,f64
23,1,3,11073,184810.9923,4.068285,4.544422,4.543365,4.068285,4.543365
23,1,3,11074,184827.5539,4.071086,4.546643,4.544422,4.071086,4.544422
23,1,3,11075,184844.3091,4.07493,4.545161,4.545161,4.07493,4.545161
23,1,3,11076,184860.9299,4.081421,4.543365,4.546643,4.080965,4.546643
23,1,3,11077,184877.6867,4.080965,4.552524,4.552524,4.081421,4.552524
23,1,3,11078,184894.3395,4.086535,4.555312,4.553684,4.086535,4.553684
…,…,…,…,…,…,…,…,…,…
513,12,50,161529,2.6960e6,4.940831,4.702713,4.665311,4.943935,4.665311
513,12,50,161530,2.6961e6,4.942234,4.707606,4.664813,4.942978,4.664813


In [31]:
from src.features.filtering import butterworth_filter

SAMPLE_RATE = 60


@map_trials
def low_pass_filter_pupil(
    df: pl.DataFrame,
    sample_rate: float = SAMPLE_RATE,
    lowcut: float | None = None,
    highcut: float | None = None,
    order: int = 2,
    pupil_columns: list[str] = ["pupil_r", "pupil_l"],
) -> pl.DataFrame:
    return df.with_columns(
        pl.col(
            pupil_columns
        ).map_batches(  # map_batches to apply the filter to each column
            lambda x: butterworth_filter(
                x,
                SAMPLE_RATE,
                lowcut=lowcut,
                highcut=highcut,
                order=order,
            )
        )
    )


smooth_pupil = low_pass_filter_pupil(median_pupil, highcut=4)

median_pupil.hvplot(
    x="timestamp", y=["pupil_r"], groupby="trial_id"
) * smooth_pupil.hvplot(x="timestamp", y=["pupil_r"], groupby="trial_id")


BokehModel(combine_events=True, render_bundle={'docs_json': {'ea81b5fc-2fdc-4ca9-b44d-9bc03687df32': {'version…